<a href="https://colab.research.google.com/github/imsandeepk/Fine_tune/blob/main/fine_tune_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget https://download.niggeshwar.workers.dev/4:/sandeep/e_OH_1.xlsx

--2023-06-09 19:50:17--  https://download.niggeshwar.workers.dev/4:/sandeep/e_OH_1.xlsx
Resolving download.niggeshwar.workers.dev (download.niggeshwar.workers.dev)... 172.67.164.115, 104.21.41.113, 2606:4700:3033::6815:2971, ...
Connecting to download.niggeshwar.workers.dev (download.niggeshwar.workers.dev)|172.67.164.115|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 259655932 (248M) [application/vnd.openxmlformats-officedocument.spreadsheetml.sheet]
Saving to: ‘e_OH_1.xlsx’

e_OH_1.xlsx         100%[===================>] 247.63M   112MB/s    in 2.2s    

2023-06-09 19:50:21 (112 MB/s) - ‘e_OH_1.xlsx’ saved [259655932/259655932]



In [ ]:
import pandas as pd
# df = pd.read_excel('/content/e_OH_1.xlsx', nrows=10000, header=None)
df = pd.read_csv('/content/final_e_OH_1.csv')

In [ ]:
intervals = range(-75, -6, 5)

def calculate_interval_frequency(row):
    frequency = []
    for interval in intervals:
        count = sum((interval <= int(value) < interval + 5) for value in row[1:])
        frequency.append(count)
    return frequency

interval_frequency = df.apply(calculate_interval_frequency, axis=1)

interval_columns = [f'({interval}, {interval + 5})' for interval in intervals]
df[interval_columns] = pd.DataFrame(interval_frequency.tolist())

df = df.drop(df.columns[1:6401], axis=1)

df = df.rename(columns={df.columns[0]: 'compound'})

In [ ]:
df_copy = df

In [ ]:
df['completion'] = df.apply(lambda row: ';'.join(f"{row[col]}" for col in df.columns[1:]), axis=1)
s = ['compound', 'completion']
final = df[s]
final = final.rename(columns={'compound': "prompt"})

In [ ]:
final

,prompt,completion
0,Fe8Mn28Co24Cu1V3,0;20;1586;3796;978;20;0;0;0;0;0;0;0;0
1,V16Fe17Co24Mn6Cu1,0;1;369;2390;2599;986;55;0;0;0;0;0;0;0
2,V8Cu21Fe3Co23Mn9,0;0;8;364;1299;983;636;433;358;492;754;829;239;5
3,V9Co29Fe15Mn6Cu5,0;0;13;456;1807;2623;1284;207;10;0;0;0;0;0
4,Fe4Cu26Co22Mn11V1,0;0;2;114;470;624;491;451;415;666;1137;1449;55...
...,...,...
3284,Mn25Cu20Fe4Co10V5,0;337;1624;1238;574;374;313;254;285;403;528;40...
3285,Fe22Cu11Co13Mn12V6,0;0;181;1229;1313;1352;1475;713;130;7;0;0;0;0
3286,Fe7Cu18V5Co22Mn12,0;0;20;666;1270;831;477;329;516;1091;1034;159;7;0
3287,Fe18Cu13Co17Mn9V7,0;0;23;748;1505;1057;1248;1123;559;136;1;0;0;0


In [ ]:
final.to_csv('prompt_completion_e_OH_1.csv', index=False)

In [ ]:
print(final.iloc[1])

prompt                                        V16Fe17Co24Mn6Cu1
completion    (-75, -70): 0; (-70, -65): 1; (-65, -60): 369;...
Name: 1, dtype: object


In [ ]:
row_sums = sum(df.apply(lambda row: row[1:][row[1:].apply(lambda x: isinstance(x, int))].sum(), axis=1))/3289
row_sums

6400.0

In [ ]:
df.to_csv('final_e_OH_1.csv', index=False)

In [ ]:
final

,prompt,completion
0,Fe8Mn28Co24Cu1V3,"(-75, -70): 0; (-70, -65): 20; (-65, -60): 158..."
1,V16Fe17Co24Mn6Cu1,"(-75, -70): 0; (-70, -65): 1; (-65, -60): 369;..."
2,V8Cu21Fe3Co23Mn9,"(-75, -70): 0; (-70, -65): 0; (-65, -60): 8; (..."
3,V9Co29Fe15Mn6Cu5,"(-75, -70): 0; (-70, -65): 0; (-65, -60): 13; ..."
4,Fe4Cu26Co22Mn11V1,"(-75, -70): 0; (-70, -65): 0; (-65, -60): 2; (..."
...,...,...
3284,Mn25Cu20Fe4Co10V5,"(-75, -70): 0; (-70, -65): 337; (-65, -60): 16..."
3285,Fe22Cu11Co13Mn12V6,"(-75, -70): 0; (-70, -65): 0; (-65, -60): 181;..."
3286,Fe7Cu18V5Co22Mn12,"(-75, -70): 0; (-70, -65): 0; (-65, -60): 20; ..."
3287,Fe18Cu13Co17Mn9V7,"(-75, -70): 0; (-70, -65): 0; (-65, -60): 23; ..."


In [ ]:
df = df.drop('completion', axis=1)

In [ ]:
final[0:650].to_json('train.jsonl', orient='records', lines=True, force_ascii=False)
final[651:].to_json('valid.jsonl', orient='records', lines=True, force_ascii=False)

In [ ]:
!openai tools fine_tunes.prepare_data -f valid.jsonl

In [ ]:
!cat '/content/valid_prepared.jsonl'

In [2]:
%env OPENAI_API_KEY=""

env: OPENAI_API_KEY=""


In [ ]:
!openai api fine_tunes.create -t "train_prepared.jsonl" -v "valid_prepared.jsonl" --compute_classification_metrics -m ada

Found potentially duplicated files with name 'train_prepared.jsonl', purpose 'fine-tune' and size 60342 bytes
file-0P9zb3BnqTpU2cbWzqD2dvOH
file-mnGnk37w94hsBQU5mgDJdbVX
Enter file ID to reuse an already uploaded file, or an empty string to upload this file anyway: 
Upload progress: 100% 60.3k/60.3k [00:00<00:00, 59.3Mit/s]
Uploaded file from train_prepared.jsonl: file-FkH3wrI5WYOSWZJrwwJrUskh
Found potentially duplicated files with name 'valid_prepared.jsonl', purpose 'fine-tune' and size 248364 bytes
file-ypg4kK0GWq8OQqBlL21weYHC
file-3zDmb1LmDgqE1uOO4Sc5tiu2
Enter file ID to reuse an already uploaded file, or an empty string to upload this file anyway: 
Upload progress: 100% 248k/248k [00:00<00:00, 273Mit/s]
Uploaded file from valid_prepared.jsonl: file-Z5SD3Cd2pGLtUF0OgyDarzkt
Error: When using "classification" as the extra_eval_type, you must specify either classification_n_classes (for multiclass classification) or classification_positive_class (for binary classification). (HTTP 

In [ ]:
!openai api fine_tunes.follow -i ft-CMlU4HOZc57oy7OmVEt5Ss5l

[2023-06-10 14:21:35] Created fine-tune: ft-CMlU4HOZc57oy7OmVEt5Ss5l
[2023-06-10 14:22:37] Fine-tune costs $0.04
[2023-06-10 14:22:38] Fine-tune enqueued. Queue number: 0
[2023-06-10 14:22:39] Fine-tune started
[2023-06-10 14:24:35] Completed epoch 1/4
[2023-06-10 14:26:16] Completed epoch 2/4
[2023-06-10 14:27:57] Completed epoch 3/4
[2023-06-10 14:29:37] Completed epoch 4/4
[2023-06-10 14:29:55] Uploaded model: ada:ft-birmingham-digital-chemistry-2023-06-10-14-29-55
[2023-06-10 14:29:56] Uploaded result file: file-yq4UrEJAzKO4sOzmdopDG779
[2023-06-10 14:29:56] Fine-tune succeeded

Job complete! Status: succeeded 🎉
Try out your fine-tuned model:

openai api completions.create -m ada:ft-birmingham-digital-chemistry-2023-06-10-14-29-55 -p <YOUR_PROMPT>


In [ ]:
!openai api fine_tunes.results -i ft-CMlU4HOZc57oy7OmVEt5Ss5l > results.csv
# results = pd.read_csv('result.csv')
# results[results['classification/accuracy'].notnull()].tail(1)

In [ ]:
import openai
openai.api_key = ''

p = f'Fe3Cu19V30Mn11Co1'

# generate the response
response = openai.Completion.create(
      engine="ada:ft-birmingham-digital-chemistry-2023-06-10-14-29-55",
      prompt=p,
      temperature=0,
      max_tokens=100,
      stop=["\n"]
    )

# grab our text from the repsonse
text = response['choices'][0]['text']


print(text+'\n')

 0;119;1274;1089;746;534;456;534;534;408;120;1;0;0

